In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [2]:
from langchain_core.output_parsers import StrOutputParser
from prompt import sundanese_overgeneration_prompt

chain = sundanese_overgeneration_prompt | model | StrOutputParser()

In [3]:
chain.invoke({"n": "1"})

TypeError: CustomExampleSelector.select_examples() takes 1 positional argument but 2 were given

In [6]:
a = [5,523, 5, 7, 8, 151]
import random

random.sample(a, 2)

[151, 5]